In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont

experiment_output_dir = "/gpfs/Labs/Uzun/DATA/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/experiments/"
experiment_name = "mESC_slow_decay_filter_ten_pct"
model_run = "chr19/model_training_001/"

image_dict = {
    "TG Prediction Accuracy": "per_gene_mean_expression.png",
    "Training R$^{2}$": "eval_results_pearson_corr.svg",
}

experiment_dir = os.path.join(experiment_output_dir, experiment_name, model_run)
pdf_path = os.path.join(experiment_dir, "experiment_results.pdf")


def to_rgb(im, bg_color=(255, 255, 255)):
    """Convert image to RGB, flattening alpha onto a solid background if needed."""
    if im.mode in ("RGBA", "LA"):
        bg = Image.new("RGB", im.size, bg_color)
        bg.paste(im, mask=im.split()[-1])
        return bg
    return im.convert("RGB")


def text_page(size, text, bg=(255, 255, 255), fg=(0, 0, 0)):
    page = Image.new("RGB", size, bg)
    draw = ImageDraw.Draw(page)
    font = ImageFont.load_default()

    x0, y0, x1, y1 = draw.textbbox((0, 0), text, font=font)
    w, h = (x1 - x0), (y1 - y0)

    draw.text(((size[0] - w) // 2, (size[1] - h) // 2), text, fill=fg, font=font)
    return page


pages = []
for title, filename in image_dict.items():
    path = os.path.join(experiment_dir, filename)
    if os.path.isfile(path):
        im = Image.open(path)
        im_rgb = to_rgb(im)

        pages.append(text_page(im_rgb.size, title))
        pages.append(im_rgb)

# Save multi-page PDF
pages[0].save(
    pdf_path,
    "PDF",
    resolution=300.0,
    save_all=True,
    append_images=pages[1:],
)

print("Wrote:", pdf_path)


UnidentifiedImageError: cannot identify image file '/gpfs/Labs/Uzun/DATA/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/experiments/mESC_slow_decay_filter_ten_pct/chr19/model_training_001/eval_results_pearson_corr.svg'